In [ ]:
import numpy as np
import os
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToPILImage
from torch.utils.data import ConcatDataset
from sklearn.metrics import accuracy_score, classification_report
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from skimage.feature import local_binary_pattern

In [ ]:
IMG_HEIGHT = 48
IMG_WIDTH = 48

# Path to the training data
TRAIN_DATA_PATH = os.path.join(os.getcwd(), '../FER with DL/data', 'train')

# Path to the test data
TEST_DATA_PATH = os.path.join(os.getcwd(), '../FER with DL/data', 'test')

# Defining transformations
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor()
])

# Load the datasets
train_dataset = ImageFolder(TRAIN_DATA_PATH, transform=transform)
test_dataset = ImageFolder(TEST_DATA_PATH, transform=transform)

# Create the dataloader for validation set only, train data still needs to be augmented
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import os
from torchvision.io import read_image

def plot_sample_images(path, n=10):
    """Plot n sample images from the given path."""
    class_folders = [os.path.join(path, f) for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
    images = []
    for folder in class_folders:
        images.extend([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')])
    if len(images) < n:
        print(f"Only found {len(images)} images in {path}.")
        n = len(images)
    fig, axs = plt.subplots(2, n//2, figsize=(15, 6))
    for i in range(n):
        img_path = images[i]
        img = read_image(img_path)
        ax = axs[i // (n//2), i % (n//2)]
        ax.imshow(img.permute(1, 2, 0).squeeze(), cmap='gray')
        ax.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

def plot_label_distribution(path):
    """Plot the distribution of labels in the given path."""
    labels = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]
    counts = [len(os.listdir(os.path.join(path, label))) for label in labels]
    plt.bar(labels, counts)
    plt.title('Distribution of Labels')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.show()

# Plot sample images and label distribution for the training data
plot_sample_images(TRAIN_DATA_PATH)
plot_label_distribution(TRAIN_DATA_PATH)

# Plot sample images and label distribution for the test data
plot_sample_images(TEST_DATA_PATH)
plot_label_distribution(TEST_DATA_PATH)

In [ ]:
# plot images from a certain emotion folder
def plot_sample_images(path, angry_files, n=10):
    """Plot n sample images from the given path."""
    # Only consider the specific folder
    class_folders = [os.path.join(path, angry_files)]
    images = []
    for folder in class_folders:
        images.extend([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')])
    if len(images) < n:
        print(f"Only found {len(images)} images in {path}.")
        n = len(images)
    fig, axs = plt.subplots(2, n//2, figsize=(15, 6))
    for i in range(n):
        img_path = images[i]
        img = read_image(img_path)
        ax = axs[i // (n//2), i % (n//2)]
        ax.imshow(img.permute(1, 2, 0).squeeze(), cmap='gray')
        ax.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

def plot_label_distribution(path, angry_files):
    """Plot the distribution of labels in the given path."""
    # Only consider the specific folder
    labels = [angry_files]
    counts = [len(os.listdir(os.path.join(path, label))) for label in labels]
    plt.bar(labels, counts)
    plt.title('Distribution of Labels')
    plt.xlabel('Label')
    plt.ylabel('Count')
    plt.show()

# Specify the folder you want to inspect
angry_files = 'happy'

# Plot sample images and label distribution for the specific folder in the training data
plot_sample_images(TRAIN_DATA_PATH, angry_files)
# plot_label_distribution(TRAIN_DATA_PATH, angry_files)

# Plot sample images and label distribution for the specific folder in the test data
# plot_sample_images(TEST_DATA_PATH, angry_files)
# plot_label_distribution(TEST_DATA_PATH, angry_files)

In [ ]:
# Check for bad data in the classes
import numpy as np
from scipy.spatial import distance

def plot_sample_images(path, specific_folder, n=10):
    """Plot n sample images from the given path."""
    # Only consider the specific folder
    class_folders = [os.path.join(path, specific_folder)]
    images = []
    for folder in class_folders:
        images.extend([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.jpg') or f.endswith('.png')])
    if len(images) < n:
        print(f"Only found {len(images)} images in {path}.")
        n = len(images)

    # Load all images and calculate the mean image
    image_data = [np.array(Image.open(img_path)) for img_path in images]
    mean_image = np.mean(image_data, axis=0)

    # Calculate the Euclidean distance from each image to the mean image
    distances = [distance.euclidean(img.flatten(), mean_image.flatten()) for img in image_data]

    # Find the n images with the largest distances
    outlier_indices = np.argsort(distances)[-n:]
    outlier_images = [images[i] for i in outlier_indices]

    # Plot the outlier images
    fig, axs = plt.subplots(2, n//2, figsize=(15, 6))
    for i, img_path in enumerate(outlier_images):
        img = read_image(img_path)
        ax = axs[i // (n//2), i % (n//2)]
        ax.imshow(img.permute(1, 2, 0).squeeze(), cmap='gray')
        ax.axis('off')
    plt.tight_layout(pad=0)
    plt.show()

# Use the function
plot_sample_images(TRAIN_DATA_PATH, 'happy')

In [ ]:
# Takes roughly 4 minutes to run this cell

# Oversampling the disgust samples since we don't have many samples

# Define additional transformations for data augmentation
augment_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor()
])

# Create a new dataset with only the "disgust" images
disgust_dataset = [img for img in train_dataset if img[1]
                   == train_dataset.class_to_idx['disgust']]

In [ ]:
from torchvision.transforms import ToTensor

# Convert Tensor to PIL Image
to_pil = ToPILImage()

# Apply data augmentation to the "disgust" images
# There are 436 disgust files in the training data so the value after range is the mutiplier
augmented_disgust_dataset = [(augment_transform(to_pil(img[0])), img[1])
                             for _ in range(8) for img in disgust_dataset]

print(type(augmented_disgust_dataset[0][0]))

In [ ]:
print(len(augmented_disgust_dataset))

In [ ]:
# Combine the original dataset with the augmented "disgust" images
train_dataset = ConcatDataset([train_dataset, augmented_disgust_dataset])

# Create the train DataLoader now that disgust images have been augmented
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Print details about the train and test datasets
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

In [ ]:
# Local Binary patterns (LBP) Feature Extraction Implementation
def extract_features_lbp(data_loader):
    # Parameters for LBP
    radius = 3
    n_points = 8 * radius
    # Initialize an empty list to store the feature vectors
    features = []
    labels = []
    # Process each batch of images
    for images, batch_labels in data_loader:
        for i in range(len(images)):
            image = images[i].numpy().squeeze()
            lbp = local_binary_pattern(image, n_points, radius)
            # Flatten the LBP image and add it to the list of feature vectors
            features.append(lbp.ravel())
            labels.append(batch_labels[i].item())
    return features, labels

In [ ]:
# Eigenfaces Feature Extraction Implmentation
def extract_features_eigenfaces(data_loader):
    # Initialize an empty list to store the images and labels
    images = []
    labels = []
    # Process each batch of images
    for batch_images, batch_labels in data_loader:
        for i in range(len(batch_images)):
            image = batch_images[i].numpy().squeeze()
            # Flatten the image and add it to the list of images
            images.append(image.ravel())
            labels.append(batch_labels[i].item())
    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    # Apply PCA to the images
    pca = PCA(n_components=150)
    features = pca.fit_transform(images)
    return features, labels

In [ ]:
# Fisherfaces Feature Extraction Implementation
def extract_features_fisherfaces(data_loader):
    # Initialize an empty list to store the images and labels
    images = []
    labels = []
    # Process each batch of images
    for batch_images, batch_labels in data_loader:
        for i in range(len(batch_images)):
            image = batch_images[i].numpy().squeeze()
            # Flatten the image and add it to the list of images
            images.append(image.ravel())
            labels.append(batch_labels[i].item())
    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)
    # Apply PCA to the images
    pca = PCA(n_components=150)
    pca_result = pca.fit_transform(images)
    # Apply LDA to the PCA result
    lda = LDA(n_components=None)
    features = lda.fit_transform(pca_result, labels)
    return features, labels

In [ ]:
# Takes roughly 4 minutes to run this cell
# Subsititute the function name based on the feature extraction method you want to use
X_train, y_train = extract_features_lbp(train_loader)

In [ ]:
# Takes roughly 4 minutes to run this cell
# Subsititute the function name based on the feature extraction method you want to use
X_test, y_test = extract_features_lbp(test_loader)

In [ ]:

print(f"Number of feature vectors: {len(X_train)}")
print(f"Shape of a feature vector: {X_train[0].shape}")


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns

def train_and_evaluate(X_train, y_train, X_test, y_test, model):
    # Create a SVM (Support Vector Machine) classifier
    # clf = KNeighborsClassifier()
    clf = model

    # Train the model with the training data and labels
    clf.fit(X_train, y_train)

    # Get the name of the model
    model_name = type(model).__name__

    # Create the filename
    filename = f'{model_name}_model.pth'

    # Save the model
    torch.save(model, filename)

    # Predict the labels of the test data
    y_pred = clf.predict(X_test)

    # Calculate the model accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate precision, recall, and F1-score
    report = classification_report(y_test, y_pred)

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    # Plot confusion matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')

    print(f"Accuracy: {accuracy * 100}%")
    print("Classification Report:")
    print(report)

In [ ]:
print("K-Nearest Neighbors Classifier")
train_and_evaluate(X_train, y_train, X_test, y_test, KNeighborsClassifier())

print("Random Forest Classifier")
train_and_evaluate(X_train, y_train, X_test, y_test, RandomForestClassifier())

print("Decision Tree Classifier")
train_and_evaluate(X_train, y_train, X_test, y_test, DecisionTreeClassifier())

print("Gaussian Naive Bayes Classifier")
train_and_evaluate(X_train, y_train, X_test, y_test, GaussianNB())

print("Support Vector Machine Classifier")
train_and_evaluate(X_train, y_train, X_test, y_test, SVC())

In [ ]:
# Displays the class labels and their corresponding indices
print(test_dataset.class_to_idx)